In [2]:
from HMM import unsupervised_HMM
from Utility import Utility
import numpy as np
import re
shakespeare = open('data/shakespeare.txt').readlines()

In [29]:
def get_sonnets(txt):
    sonnets = []
    j = 1
    song = []
    for i in range(len(txt)):
        line = txt[i]
        arr = [str(j)]
        empt = []
        if line.split() == arr:
            if song != []:
                sonnets.append(song)
            song = []
            j += 1
        else:
            if line != '\n':
                if line[0] == line[1] and line[0] == ' ':
                    song.append(line[2:len(line)])
                else:
                    song.append(line)

    rawtxt = ''
    for sonnet in sonnets:
        for line in sonnet:
            rawtxt = rawtxt + line
    i = 0
    words = {}
    for sonnet in sonnets:
        for line in sonnet:
            line = re.sub('[;,:?!.\(\)]', '', line).lower()
            for word in line.split():
                if (word not in words.keys()):
                    words[word] = i
                    i += 1
    nums = []
    for sonnet in sonnets:
        tmp = []
        for line in sonnet:
            line = re.sub('[;,:?!.\(\)]', '', line).lower()
            for word in line.split():
                tmp.append(words[word])
        nums.append(tmp)

    return nums, words

In [31]:
def unsupervised_learning2(n_states, iters, g, gmap):
    '''
    Trains an HMM using supervised learning on the file 'ron.txt' and
    prints the results.

    Arguments:
        n_states:   Number of hidden states that the HMM should have.
    '''
    genres, genre_map = g, gmap
    # Train the HMM.        
    HMM = unsupervised_HMM(genres, n_states, iters)

    # Print the transition matrix.
    print("Transition Matrix:")
    print('#' * 70)
    for i in range(len(HMM.A)):
        print(''.join("{:<12.3e}".format(HMM.A[i][j]) for j in range(len(HMM.A[i]))))
    print('')
    print('')

    # Print the observation matrix. 
    print("Observation Matrix:  ")
    print('#' * 70)
    for i in range(len(HMM.O)):
        print(''.join("{:<12.3e}".format(HMM.O[i][j]) for j in range(len(HMM.O[i]))))
    print('')
    print('')
    return HMM.A, HMM.O

In [32]:
print('')
print('')
print('#' * 70)
print("{:^70}".format("Running Code For Question 2D"))
print('#' * 70)
print('')
print('')
g, gmap = get_sonnets(shakespeare)
A1, O1 = unsupervised_learning2(8, 100, g, gmap)



######################################################################
                     Running Code For Question 2D                     
######################################################################


Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90
Iteration: 100
Transition Matrix:
######################################################################
6.572e-02   7.657e-01   4.320e-04   1.563e-07   1.045e-01   6.670e-27   6.372e-02   2.070e-10   
4.781e-01   1.613e-05   2.185e-01   6.675e-11   8.039e-20   6.813e-23   9.987e-02   2.035e-01   
8.325e-15   5.568e-03   3.596e-04   1.399e-14   3.379e-23   3.124e-01   3.455e-07   6.817e-01   
3.510e-25   1.458e-15   2.022e-01   3.784e-24   7.434e-01   9.289e-19   1.096e-02   4.345e-02   
3.410e-17   1.822e-01   3.349e-01   3.261e-01   1.568e-01   2.891e-15   3.824e-12   3.699e-15   
8.568e-36   6.666e-01   3.270e-01   3.272e-22   5.042e-03   1.394e-

In [34]:
from HMM import HiddenMarkovModel
from Utility import Utility
import random
import nltk
nltk.download('cmudict')
from nltk.corpus import cmudict 
d = cmudict.dict()
inv_map = {v: k for k, v in gmap.items()}
    
def rhyme(inp, level):
    entries = nltk.corpus.cmudict.entries()
    syllables = [(word, syl) for word, syl in entries if word == inp]
    rhymes = []
    for (word, syllable) in syllables:
         rhymes += [word for word, pron in entries if pron[-level:] == syllable[-level:]]
    return set(rhymes)


def generate_emission1(self, M, rhymer):
        '''
        Generates an emission of length M, assuming that the starting state
        is chosen uniformly at random. 

        Arguments:
            M:          Length of the emission to generate.

        Returns:
            emission:   The randomly generated emission as a list.

            states:     The randomly generated states as a list.
        '''

        emission = []
        states = []

        state_idx = random.choice(list(range(self.L)))
        emission.append(np.random.choice(list(range(self.D)), p = self.O[state_idx]))
        states.append(state_idx)
        
        syls = []
        
        for i in range(M):
            tmp = np.random.choice(range(self.D), p = self.O[state_idx])
            tmpl = list(range(self.L))
            if(i == M - 1 and rhymer == ''):
                while True:
                    tmp = np.random.choice(range(self.D), p = self.O[state_idx])
                    if(len(rhyme(inv_map[tmp], 1)) > 10):
                        break
            elif(i == M - 1 and rhymer != ''):
                while True:
                    tmp = np.random.choice(range(self.D), p = self.O[state_idx])
                    if(inv_map[tmp] in rhyme(rhymer, 1) and inv_map[tmp] != rhymer):
                        break
            emission.append(tmp)
            state_idx = np.random.choice(tmpl, p = self.A[state_idx])
            states.append(state_idx)

        return emission, states
    

def sequence_generator(A, O, k, M):
    '''
    Generates k emissions of length M using the HMM stored in the file
    'sequence_data<n>.txt' for a given n and prints the results.

    Arguments:
        N:          File index.
        K:          Number of sequences to generate.
        M:          Length of emission to generate.
    '''
    # Generate k input sequences.
    rhymer1 = ''
    rhymer2 = ''
    rhymer3 = ''
    
    for i in range(k):
        # Initialize an HMM.
        HMM = HiddenMarkovModel(A, O)
        count1 = 0
        count2 = 0
        if(i % 2 == 1 and i < k - 2):
            # Generate a single input sequence of length m.
            emission, states = generate_emission1(HMM, M, rhymer1)
            x = ' '.join([inv_map[i] for i in emission])
            rhymer1 = inv_map[emission[-1]]
            count1 += 1
            if(count1 == 2):
                rhymer1 = ''
        elif(i % 2 == 0 and i < k -2):
            # Generate a single input sequence of length m.
            emission, states = generate_emission1(HMM, M, rhymer2)
            x = ' '.join([inv_map[i] for i in emission])
            rhymer2 = inv_map[emission[-1]]
            count2 += 1
            if(count2 == 2):
                rhymer2 = ''
        elif(i >= k - 2):
            # Generate a single input sequence of length m.
            emission, states = generate_emission1(HMM, M, rhymer3)
            x = ' '.join([inv_map[i] for i in emission])
            rhymer3 = inv_map[emission[-1]]
        
        # Print the results.
        print("{:30}".format(x))

    print('')
    print('')


sequence_generator(A1, O1, 14, 10)

[nltk_data] Downloading package cmudict to C:\Users\Jen
[nltk_data]     Yu\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


decay where you what world even thee what of expiate being
leave as beauty heavily is barren and me doth thou rage
frown'st a each the appear so blood and thou and sparkling
shall thou true fairest rents happy be i will long strange
says war him i every argument though love fear to wanting
fear fled i glory happy familiar again have in this large
all supposed with was aught if second yet pride and loving
face beauty's as flies and that in all in spirit change
that cruel in love but me soul heretic come me paying
thou nor my love in thy number he why when age
night hence since groan an new-appearing miles voice ere leaves nothing
every an thou to mistaking the muse and this is change
are love never thou divine i love robbing from such thy
have be zealous of the nature's that the for name eye




In [39]:
import pyphen
dic = pyphen.Pyphen(lang='en')

def count_syllables(word):
    return dic.inserted(word).count('-')+1

def generate_emission2(self, M):
        '''
        Generates an emission of length M, assuming that the starting state
        is chosen uniformly at random. 

        Arguments:
            M:          Length of the emission to generate.

        Returns:
            emission:   The randomly generated emission as a list.

            states:     The randomly generated states as a list.
        '''

        emission = []
        states = []

        state_idx = random.choice(list(range(self.L)))
        emission.append(np.random.choice(list(range(self.D)), p = self.O[state_idx]))
        states.append(state_idx)
        syllables = 0
        
        while True:
            tmp = np.random.choice(range(self.D), p = self.O[state_idx])
            while(syllables + count_syllables(inv_map[tmp]) > M):
                tmp = np.random.choice(range(self.D), p = self.O[state_idx])
            emission.append(tmp)
            state_idx = np.random.choice(list(range(self.L)), p = self.A[state_idx])
            states.append(state_idx)
            syllables += count_syllables(inv_map[tmp])
            if(syllables == M):
                break
        return emission, states
    
def sequence_generator2(A, O, k, M):
    '''
    Generates k emissions of length M using the HMM stored in the file
    'sequence_data<n>.txt' for a given n and prints the results.

    Arguments:
        N:          File index.
        K:          Number of sequences to generate.
        M:          Length of emission to generate.
    '''


    # Generate k input sequences.
    for i in range(k):
        # Initialize an HMM.
        HMM = HiddenMarkovModel(A, O)

        # Generate a single input sequence of length m.
        emission, states = generate_emission2(HMM, M)
        x = ' '.join([inv_map[i] for i in emission])
        
        # Print the results.
        print("{:30}".format(x))

    print('')
    print('')



sequence_generator2(A1, O1, 14, 10)

it certain and no storm-beaten shadow
to to evermore that their too my his when
which nurse do so doth have mistress' depart all
tender beauty vex aye is be his from which from
steeled be doth fadeth knows minds like whose earth my by
bad aid makes live what stands yet with dwell in more
mind's eye i haste so heaven child is me might
me hear find tendered more pencil temperate
to with have soft should blessing day and disdain
deepest as shame before compounds thy i my self
o thou what in hate found all my their compare
and with forgetful my night i teachest your
by by live to you small it my sad muse be
the mine that bear did make sequent change time watch




In [ ]:
import os
import numpy as np
from IPython.display import HTML

from HMM import unsupervised_HMM
from HMM_helper import (
    text_to_wordcloud,
    states_to_wordclouds,
    parse_observations,
    sample_sentence,
    visualize_sparsities,
    animate_emission
)

txtflat = ''
for liste in txt:
    txtflat += liste

obs, obs_map = parse_observations(txtflat)
wordclouds = states_to_wordclouds(HMM, obs_map)